In [62]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# File system manangement
import os

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# modeling
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn_pandas import DataFrameMapper

from sklearn.model_selection import *
import sklearn.metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import RFECV
from sklearn.metrics import fbeta_score, make_scorer

#explainability
import shap

#serialization
import joblib

In [63]:
#read the data 
app_train = pd.read_csv("../raw_data/application_train.csv").astype("object")


# Features Types

We need to convert variables that are not correctly cast as of float in the raw data : 

In [64]:
def convert_dtypes_func(df):
    #List the columns to be converted to float
    list_columns_to_convert_to_float = ("CNT_CHILDREN" 
                                ,"AMT_INCOME_TOTAL" 
                                ,"AMT_CREDIT" 
                                ,"AMT_ANNUITY" 
                                ,"AMT_GOODS_PRICE" 
                                ,"DAYS_BIRTH"
                                ,"DAYS_EMPLOYED" 
                                ,"DAYS_REGISTRATION" 
                                ,"DAYS_ID_PUBLISH"
                                ,"OWN_CAR_AGE"
                                ,"EXT_SOURCE_1               "
                                ,"EXT_SOURCE_2               "
                                ,"EXT_SOURCE_3               "
                                ,"APARTMENTS_AVG             "
                                ,"BASEMENTAREA_AVG           "
                                ,"YEARS_BEGINEXPLUATATION_AVG"
                                ,"YEARS_BUILD_AVG            "
                                ,"COMMONAREA_AVG             "
                                ,"ELEVATORS_AVG              "
                                ,"ENTRANCES_AVG              "
                                ,"FLOORSMAX_AVG              "
                                ,"FLOORSMIN_AVG              "
                                ,"LANDAREA_AVG               "
                                ,"LIVINGAPARTMENTS_AVG       "
                                ,"LIVINGAREA_AVG             "
                                ,"NONLIVINGAPARTMENTS_AVG    "
                                ,"NONLIVINGAREA_AVG          "
                                ,"APARTMENTS_MODE            "
                                ,"BASEMENTAREA_MODE          "
                                ,"YEARS_BEGINEXPLUATATION_MODE"
                                ,"YEARS_BUILD_MODE           "
                                ,"COMMONAREA_MODE            "
                                ,"ELEVATORS_MODE             "
                                ,"ENTRANCES_MODE             "
                                ,"FLOORSMAX_MODE             "
                                ,"FLOORSMIN_MODE             "
                                ,"LANDAREA_MODE              "
                                ,"LIVINGAPARTMENTS_MODE      "
                                ,"LIVINGAREA_MODE            "
                                ,"NONLIVINGAPARTMENTS_MODE   "
                                ,"NONLIVINGAREA_MODE         "
                                ,"APARTMENTS_MEDI            "
                                ,"BASEMENTAREA_MEDI          "
                                ,"YEARS_BEGINEXPLUATATION_MEDI"
                                ,"YEARS_BUILD_MEDI           "
                                ,"COMMONAREA_MEDI            "
                                ,"ELEVATORS_MEDI             "
                                ,"ENTRANCES_MEDI             "
                                ,"FLOORSMAX_MEDI             "
                                ,"FLOORSMIN_MEDI             "
                                ,"LANDAREA_MEDI              "
                                ,"LIVINGAPARTMENTS_MEDI      "
                                ,"LIVINGAREA_MEDI            "
                                ,"NONLIVINGAPARTMENTS_MEDI   "
                                ,"NONLIVINGAREA_MEDI         "
                                ,"TOTALAREA_MODE             "
                                ,"OBS_30_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_30_CNT_SOCIAL_CIRCLE   "
                                ,"OBS_60_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_60_CNT_SOCIAL_CIRCLE   "
                                ,"DAYS_LAST_PHONE_CHANGE     "
                                ,"AMT_REQ_CREDIT_BUREAU_HOUR"
                                ,"AMT_REQ_CREDIT_BUREAU_DAY "
                                ,"AMT_REQ_CREDIT_BUREAU_WEEK"
                                ,"AMT_REQ_CREDIT_BUREAU_MON "
                                ,"AMT_REQ_CREDIT_BUREAU_QRT "
                                ,"AMT_REQ_CREDIT_BUREAU_YEAR")       

    #remove spaces in the list created
    list_columns_to_convert_to_float = [s.strip() for s in list_columns_to_convert_to_float]   
    
    convert_count = 0
    # convert_count_2 = 0

    #convert object columns to float
    for col in list_columns_to_convert_to_float:
        
        df[col] = df[col].astype(float)
        
        # Keep track of how many columns were label encoded
        convert_count += 1

    print('%d object columns were converted to float.' % convert_count)
    
    return df

In [65]:
app_train = convert_dtypes_func(app_train)

67 object columns were converted to float.


# Feature Engineering

In [66]:
def feature_eng(df):

    # Create an anomalous flag column
    df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243

    # Replace the anomalous values with nan
    df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

    #correct sign of Days Birth
    df["DAYS_BIRTH"] = abs(df["DAYS_BIRTH"])
    df['AGE_INT'] = (df['DAYS_BIRTH'] / 365).astype(float)

    #footing financial ratios
    df['annuity_income_ratio'] = df['AMT_INCOME_TOTAL'] / df['AMT_ANNUITY'] 
    df['credit_annuity_ratio'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['credit_goods_price_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['credit_downpayment'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']
 
    print('Feature engineering success')
   
    return df

In [67]:
app_train = feature_eng(app_train)

Feature engineering success


# Feature Selection

In [68]:
def df_feature_selection(df):
    columns_list = df.columns.to_list()

    columns_to_drop_intersec = (['DAYS_BIRTH',
    'REGION_RATING_CLIENT_W_CITY',
    'BASEMENTAREA_MODE',
    'YEARS_BUILD_MODE',
    'COMMONAREA_MODE',
    'ELEVATORS_MODE',
    'ENTRANCES_MODE',
    'FLOORSMAX_MODE',
    'LANDAREA_MODE',
    'LIVINGAPARTMENTS_MODE',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAREA_MODE',
    'APARTMENTS_MEDI',
    'BASEMENTAREA_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI',
    'COMMONAREA_MEDI',
    'ELEVATORS_MEDI',
    'ENTRANCES_MEDI',
    'FLOORSMAX_MEDI',
    'FLOORSMIN_MEDI',
    'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI',
    'NONLIVINGAREA_MEDI',
    'OBS_60_CNT_SOCIAL_CIRCLE'])

    #Drop columns
    df_selec_col = pd.DataFrame(data=df, columns=columns_list).drop(columns_to_drop_intersec, axis=1)
    
    print('Feature selection success')

    return df_selec_col

In [69]:
app_train = df_feature_selection(app_train)

Feature selection success


# Encoding - Standardization - Imputation

In [72]:
#keep id into a separate serie
user_id = app_train[['SK_ID_CURR']]

#create list of features by type
cat_features = app_train.select_dtypes(include=['object']).drop(['SK_ID_CURR','TARGET'], axis=1)
num_features = app_train.select_dtypes(exclude=['object'])

#One hot encoding categorical variables
ohe = OneHotEncoder()
cat_array = ohe.fit_transform(cat_features).todense()
cat_array = np.asarray(cat_array)

#Mean Imputation of missing values
imp_mean = SimpleImputer()
num_array = num_features.to_numpy()
num_array = imp_mean.fit_transform(num_array)

#Standard Scaling numerical variables
scaler = StandardScaler()
num_array = scaler.fit_transform(num_array)

#concatenate
X = np.concatenate([cat_array, num_array], axis=1)
y = app_train['TARGET'].astype(float)

#dataframe
df_train = pd.concat([user_id, cat_features, num_features], axis=1)
df_train = df_train.astype('object')


In [ ]:
#Model with best params
model = LogisticRegression(class_weight = 'balanced', C=0.005, max_iter=1000)

#fit
model.fit(X,y)

# Serialization

In [ ]:
joblib.dump(ohe, 'bin/ohe.joblib') #into a folder bin (for binary)
joblib.dump(imp_mean, 'bin/imp_mean.joblib')
joblib.dump(scaler, 'bin/scaler.joblib')
joblib.dump(model, 'bin/model.joblib')

['bin/model.joblib']

# Export DataSet

Export pre-processed dataset to be used for Dashboard Streamlit (on which heroku deployed model will predicting scoring) : 

In [60]:
app_test = pd.read_csv("../raw_data/application_test.csv").astype("object")

app_test_converted = convert_dtypes_func(app_test)
app_test_eng = feature_eng(app_test)
app_test_selec_col = df_feature_selection(app_test_eng)

df_test = app_test_selec_col.sample(frac=0.05, random_state=0)
df_test = df_test.astype('object')

df_test.to_csv("./dashboard_data/df_test.csv", index=False)

print('df_test exported to dashboard_data folder.')


67 object columns were converted to float.
Feature engineering success
Feature selection success
df_test exported to dashboard_data folder.


# Pydantic Documentation

In [25]:
from pydantic import BaseModel, create_model, Field, ValidationError

model = {}
for i in range(len(df_train.dtypes)):
    name = df_train.dtypes.index[i]
    var_type = type(df_train.iloc[0, i])
    model.update({name: (var_type, Field(...))})

In [27]:
data_dict = joblib.dump(model, 'bin/data_dict.joblib')

['bin/data_dict.joblib']

In [102]:
colonnes_pandas_profiling = ["CODE_GENDER",
                             "AGE_INT", 
                             "NAME_TYPE_SUITE",
                             "NAME_EDUCATION_TYPE",
                             "NAME_INCOME_TYPE",
                             "ORGANIZATION_TYPE",
                             "OCCUPATION_TYPE",
                             "NAME_HOUSING_TYPE",
                             "CNT_CHILDREN", 
                             "AMT_INCOME_TOTAL", 
                             "AMT_GOODS_PRICE"                             
                             ]

colonnes_scoring_predicted = ["CODE_GENDER",
                              "NAME_EDUCATION_TYPE",
                              "DAYS_EMPLOYED"
                              "EXT_SOURCE_1",
                              "EXT_SOURCE_2",
                              "EXT_SOURCE_3",
                              "credit_downpayment",
                              "AMT_INCOME_TOTAL",
                              "annuity_income_ratio",
                              ]

In [118]:
liste = [i for i in df_test.columns.to_list()]
matching = [s for s in liste if "AGE" in s]
matching

['OWN_CAR_AGE', 'AGE_INT']

In [112]:
liste_value = df_test['CODE_GENDER'].value_counts().tolist()
liste_value

[1613, 824]

In [115]:
df_test['YEARS_BIRTH']

KeyError: 'YEARS_BIRTH'